In [34]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_1430/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [35]:
import csv
import collections
import sys
import pickle
import random 
import copy

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as transforms

from tqdm import tqdm
from PIL import Image

In [36]:
sys.path.append('../')

from utils.dataset import img_float2int, feature_names, label_names

from utils.transformations import SimCLR_augmentation
from utils.transformations import Astro_augmentation
from utils.transformations import Astro_augmentation_v2
from utils.transformations import Astro_augmentation_v3
from utils.transformations import Jitter_astro
from utils.transformations import Jitter_simclr
from utils.transformations import Crop_astro
from utils.transformations import Crop_simclr
from utils.transformations import Rotation
from utils.transformations import Rotation_v2
from utils.transformations import Gaussian_blur
from utils.transformations import Resize_img
from utils.transformations import RandomPerspective
from utils.transformations import GridDistortion
from utils.transformations import ElasticTransform

# Función que explora el dataset

In [5]:
# Print the type of the objects inside the input.
def dataset_structure(data, space=''):

    # Iteration along dictionary
    for keys in data:

        # If the object is other dictionary
        if(type(data[keys]) is dict):

            print('{0}-{1}:'.format(space,keys))
            dataset_structure(data[keys], space + '\t')

        else:

            # Type of the object
            type_obj = type(data[keys]).__name__

            # Extracts an example of the elements into the object
            example = data[keys][0]
            type_example = type(example).__name__

            # If the element is an ndarray, its shape is printed
            if(type_example=='ndarray'):
                shape_example = str(np.shape(example))
            else:
                shape_example = 'Non-ndarray'

            # Print the information (length and type) of the structure inside
            # the dictionary
            print("{0}-{1:<10}N={2:<10}Type:{3}".format(space, keys, len(data[keys]), type_obj), end="")
            print("\t-->\tType:{1:<10}Shape:{2:<10}".format(space, type_example, shape_example))

# Estructura del dataset

In [5]:
# Dataset is loaded
with open('../dataset/td_ztf_stamp_17_06_20.pkl', 'rb') as f:
    data = pickle.load(f)

# Inner structure of dataset
dataset_structure(data)

-Train:
	-images    N=72710     Type:ndarray	-->	Type:ndarray   Shape:(63, 63, 3)
	-labels    N=72710     Type:ndarray	-->	Type:int64     Shape:Non-ndarray
	-features  N=72710     Type:ndarray	-->	Type:ndarray   Shape:(26,)     
-Validation:
	-images    N=500       Type:ndarray	-->	Type:ndarray   Shape:(63, 63, 3)
	-labels    N=500       Type:ndarray	-->	Type:int64     Shape:Non-ndarray
	-features  N=500       Type:ndarray	-->	Type:ndarray   Shape:(26,)     
-Test:
	-images    N=500       Type:ndarray	-->	Type:ndarray   Shape:(63, 63, 3)
	-labels    N=500       Type:ndarray	-->	Type:int64     Shape:Non-ndarray
	-features  N=500       Type:ndarray	-->	Type:ndarray   Shape:(26,)     


# Reduce dataset

In [6]:
new_data = {}

for subset in ['Train', 'Validation', 'Test']:
    
    new_data[subset] = {'images':[], 'features':[], 'labels':[]}
    L = len(data[subset]['labels'])
    idx = 0
    
    rep_list = []    
    
    for i in tqdm(range(L)):
        if (i not in rep_list):
            for j in range(i+1, L):
                #if (j not in rep_list):
                if (np.linalg.norm(data[subset]['images'][j] - data[subset]['images'][i]) < 0.00001):
                    rep_list.append(j)
    
        

        if (i not in rep_list):
            new_data[subset]['images'].append(data[subset]['images'][i])
            new_data[subset]['features'].append(data[subset]['features'][i])
            new_data[subset]['labels'].append(data[subset]['labels'][i])
        
            idx += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 445.63it/s]


# Guarda el dataset

In [6]:
# Save dataset        
file = open('td_ztf_stamp_17_06_20_red.pkl', 'wb')
pickle.dump(new_data, file)
file.close()

NameError: name 'new_data' is not defined

In [8]:
def class_dataset(data):
    
    counter_train = collections.Counter(data['Train']['labels'])
    counter_validation = collections.Counter(data['Validation']['labels'])
    counter_test = collections.Counter(data['Test']['labels'])

    print('Train:')
    for key, value in counter_train.items():
        print(f'\t{key}:{value}')
    
    print('Validation:')
    for key, value in counter_validation.items():
        print(f'\t{key}:{value}')

    print('Test:')
    for key, value in counter_test.items():
        print(f'\t{key}:{value}')

# Separa dataset para SimCLR

In [21]:
with open('../dataset/td_ztf_stamp_17_06_20_red.pkl', 'rb') as f:
    data = pickle.load(f)

In [22]:
class_dataset(data)

Train:
	2:14517
	0:14542
	1:1396
	4:10528
	3:9599
Validation:
	0:100
	1:100
	2:100
	3:100
	4:100
Test:
	0:100
	1:100
	2:100
	3:100
	4:100


In [23]:
def partion_dic(data, p):
    
    partion = {}
    N = len(data['labels'])

    idx = random.sample(range(N), int(N*p))
    
    for key, val in data.items():
        partion[key] = [data[key][i] for i in idx]
    
    return partion

In [24]:
def split_dic(data, p):
    
    partion_1 = {}
    partion_2 = {}
    
    N = len(data['labels'])
    idx = random.sample(range(N), int(N*p))
    not_idx = np.delete(np.arange(N), idx)
    
    for key, val in data.items():
        partion_1[key] = [data[key][i] for i in idx]
        partion_2[key] = [data[key][i] for i in not_idx]
        
    
    return partion_1, partion_2

In [29]:
def balance(data):
    
    N = len(data['labels'])
    balanced_data = data
    counter = collections.Counter(data['labels'])
    
    deficit = {key: max(counter.values()) - val for key, val in counter.items()}

    for key, value in counter.items():
        print(f'\t{key}:{value}')
       
    
    for key, val in deficit.items():
        
        idx = np.random.choice([i for i in range(N) if key==data['labels'][i]], val)
        
        balanced_data['images'] += [data['images'][i] for i in idx]
        balanced_data['features'] += [data['features'][i] for i in idx]
        balanced_data['labels'] += [data['labels'][i] for i in idx]
        
    return balanced_data

In [30]:
data_sup_100 = {'Train': balance(copy.deepcopy(data['Train'])), 'Validation': data['Validation'], 'Test': data['Test']}
class_dataset(data_sup_100)

# Save dataset        
file = open('td_ztf_stamp_17_06_20_sup_100.pkl', 'wb')
pickle.dump(data_sup_100, file)
file.close()

	2:14517
	0:14542
	1:1396
	4:10528
	3:9599
Train:
	2:14542
	0:14542
	1:14542
	4:14542
	3:14542
Validation:
	0:100
	1:100
	2:100
	3:100
	4:100
Test:
	0:100
	1:100
	2:100
	3:100
	4:100


In [31]:
p = 0.1
data_sup_10 = {'Train': balance(copy.deepcopy(partion_dic(data['Train'], p))), 'Validation': data['Validation'], 'Test': data['Test']}

class_dataset(data_sup_10)

# Save dataset        
file = open('td_ztf_stamp_17_06_20_sup_10.pkl', 'wb')
pickle.dump(data_sup_10, file)
file.close()

	0:1496
	4:1086
	3:905
	2:1428
	1:143
Train:
	0:1496
	4:1496
	3:1496
	2:1496
	1:1496
Validation:
	0:100
	1:100
	2:100
	3:100
	4:100
Test:
	0:100
	1:100
	2:100
	3:100
	4:100


In [33]:
p = 0.01
data_sup_1 = {'Train': balance(copy.deepcopy(partion_dic(data['Train'], p))), 'Validation': data['Validation'], 'Test': data['Test']}
class_dataset(data_sup_1)

# Save dataset        
file = open('td_ztf_stamp_17_06_20_sup_1.pkl', 'wb')
pickle.dump(data_sup_1, file)
file.close()

	4:110
	2:162
	3:73
	0:146
	1:14
Train:
	4:162
	2:162
	3:162
	0:162
	1:162
Validation:
	0:100
	1:100
	2:100
	3:100
	4:100
Test:
	0:100
	1:100
	2:100
	3:100
	4:100
